In [42]:
import json
from typing import List, Dict, Tuple, Optional
from mingus.core import scales, notes, chords
import enum
import itertools
import numpy as np
from dataclasses import dataclass

class Interval(enum.Enum):
    UNISON = 0
    MINOR_SECOND = 1
    MAJOR_SECOND = 2
    MINOR_THIRD = 3
    MAJOR_THIRD = 4
    PERFECT_FOURTH = 5
    TRITONE = 6
    PERFECT_FIFTH = 7
    MINOR_SIXTH = 8
    MAJOR_SIXTH = 9
    MINOR_SEVENTH = 10
    MAJOR_SEVENTH = 11

class ChordQuality(enum.Enum):
    MAJOR = (Interval.UNISON, Interval.MAJOR_THIRD, Interval.PERFECT_FIFTH)
    MINOR = (Interval.UNISON, Interval.MINOR_THIRD, Interval.PERFECT_FIFTH)
    DIMINISHED = (Interval.UNISON, Interval.MINOR_THIRD, Interval.TRITONE)
    AUGMENTED = (Interval.UNISON, Interval.MAJOR_THIRD, Interval.MINOR_SIXTH)

class Note:
    def __init__(self, name: str, octave: int):
        self.name = name
        self.octave = octave

    def __str__(self):
        return f'{self.name}{self.octave}'

    def __lt__(self, other):
        notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
        if self.octave != other.octave:
            return self.octave < other.octave
        return notes.index(self.name) < notes.index(other.name)

class Fretboard:
    def __init__(self, tuning):
        self.tuning = tuning
        self.create_fretboard()

    def create_fretboard(self):
        notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
        num_strings = len(self.tuning)
        num_frets = 12  # You can adjust this if needed
        self.fretboard = np.empty((num_strings, num_frets), dtype=object)
        for i, string in enumerate(self.tuning):
            start_index = notes.index(string.name)
            for fret in range(num_frets):
                note_index = (start_index + fret) % 12
                octave = string.octave + (start_index + fret) // 12
                self.fretboard[i][fret] = Note(notes[note_index], octave)

    def print_fretboard(self):
        print("Fretboard:")
        print(" ", end="")
        for fret in range(12):
            print(f"{fret:2d} ", end="")
        print()
        for string, tuning_note in enumerate(self.tuning):
            print(f"{tuning_note}: ", end="")
            for fret in range(12):
                note = self.fretboard[string][fret]
                print(f"{note.name:2s} ", end="")
            print()

    def get_pitch(self, string: int, fret: int) -> Note:
        return self.fretboard[string][fret]


@dataclass
class ChordDiagram:
    frets: List[int]
    fingers: List[int]
    mutes: List[bool]
    root: str
    quality: ChordQuality
    fretboard: Fretboard

    def __post_init__(self):
        self.notes = self.get_notes_with_pitches()
        self.root_note = self.find_root_note()
        self.bass_note = self.notes[0] if self.notes else None
        self.inversion = self.find_inversion()
        self.is_partial = len(self.notes) < 3  # Assuming a triad

    def get_notes_with_pitches(self):
        return [self.fretboard.get_pitch(string, fret) for string, fret in enumerate(self.frets) if fret >= 0]

    def find_root_note(self):
        for note in self.notes:
            if note.name == self.root:
                return note
        return None

    def get_unique_id(self):
        return tuple(self.frets)

    def get_position(self):
        played_frets = [f for f in self.frets if f >= 0]
        if not played_frets:
            return "Open"
        min_fret = min(played_frets)
        max_fret = max(played_frets)
        if min_fret == 0:
            return "Open"
        elif min_fret == max_fret and self.fingers.count(1) >= 3:
            return f"Barre at fret {min_fret}"
        else:
            return f"Fret {min_fret}"

    def get_notes_with_strings(self):
        return [f"{self.fretboard.tuning[i].name}{self.fretboard.tuning[i].octave}:{note}" for i, note in enumerate(self.notes)]

    def find_inversion(self):
        if not self.root_note or not self.notes:
            return 0
        return self.notes.index(self.root_note)

    def get_chord_name(self):
        inversion_names = ['', ' (first inversion)', ' (second inversion)']
        return f"{self.root} {self.quality.name}{inversion_names[self.inversion]}"

    def __lt__(self, other):
        return self.bass_note < other.bass_note if self.bass_note and other.bass_note else False


class InstrumentChordGenerator:
    def __init__(self, tuning=['G3', 'D4', 'A4', 'E5']):  
        self.tuning = [Note(name=note[:-1], octave=int(note[-1])) for note in tuning]
        self.fretboard = Fretboard(self.tuning)

    def generate_all_triads(self, root: str):
        triads = []
        qualities = [ChordQuality.MAJOR, ChordQuality.MINOR, ChordQuality.DIMINISHED, ChordQuality.AUGMENTED]
        for quality in qualities:
            triads.extend(self.generate_chord(root, quality))
        return triads

    def generate_chord(self, root: str, quality: ChordQuality, full_chord=True, partial_chord=True, inversions=True, open_strings=True, max_fret_span=4) -> List[ChordDiagram]:
        chord_notes = self._get_chord_notes(root, quality)
        shapes = self._build_shape(chord_notes, full_chord, partial_chord, inversions, open_strings, max_fret_span)
        unique_diagrams = {}
        for shape in shapes:
            diagram = self._create_chord_diagram(shape, root, quality)
            unique_id = diagram.get_unique_id()
            if unique_id not in unique_diagrams:
                unique_diagrams[unique_id] = diagram
        return list(unique_diagrams.values())

    def _create_chord_diagram(self, shape, root: str, quality):
        frets = [-1] * 4
        fingers = [0] * 4
        mutes = [True] * 4
        for string, fret in shape:
            frets[string] = fret
            fingers[string] = 1 if fret > 0 else 0
            mutes[string] = False
        return ChordDiagram(frets, fingers, mutes, root, quality, self.fretboard)

    def _get_chord_notes(self, root: str, quality: ChordQuality) -> List[str]:
        notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
        root_index = notes.index(root)
        return [notes[(root_index + interval.value) % 12] for interval in quality.value]

    def _build_shape(self, chord_notes: List[str], full_chord=True, partial_chord=True, inversions=True, open_strings=True, max_fret_span=4) -> List[List[Tuple[int, int]]]:
        shapes = []
        if full_chord:
            full_shapes = self._find_full_chord(chord_notes, open_strings, max_fret_span)
            shapes.extend(full_shapes)
        if partial_chord:
            partial_shapes = self._find_partial_chord(chord_notes, open_strings, max_fret_span)
            shapes.extend(partial_shapes)
        if inversions:
            inversion_shapes = self._find_inversions(chord_notes, open_strings, max_fret_span)
            shapes.extend(inversion_shapes)
        return shapes

    def _find_full_chord(self, chord_notes: List[str], open_strings: bool, max_fret_span: int) -> List[List[Tuple[int, int]]]:
        shapes = []
        for combo in itertools.combinations(range(4), len(chord_notes)):
            shape = []
            for i, string in enumerate(combo):
                if open_strings and self.fretboard.get_pitch(string, 0).name == chord_notes[i]:
                    shape.append((string, 0))
                else:
                    for fret in range(12):  # Search all frets up to 12
                        if self.fretboard.get_pitch(string, fret).name == chord_notes[i]:
                            shape.append((string, fret))
                            break
            if len(shape) == len(chord_notes) and self._is_valid_shape(shape, max_fret_span):
                shapes.append(shape)
        return shapes

    def _find_partial_chord(self, chord_notes: List[str], open_strings: bool, max_fret_span: int) -> List[List[Tuple[int, int]]]:
        shapes = []
        for r in range(1, len(chord_notes)):  
            for subset in itertools.combinations(chord_notes, r):
                shapes.extend(self._find_full_chord(subset, open_strings, max_fret_span))
        return shapes

    def _find_inversions(self, chord_notes: List[str], open_strings: bool, max_fret_span: int) -> List[List[Tuple[int, int]]]:
        shapes = []
        for inversion in itertools.permutations(chord_notes):
            shapes.extend(self._find_full_chord(inversion, open_strings, max_fret_span))
        return shapes

    def _is_valid_shape(self, shape: List[Tuple[int, int]], max_fret_span: int) -> bool:
        frets = [fret for _, fret in shape if fret > 0]
        return not frets or max(frets) - min(frets) <= max_fret_span



class LocalChordGenerator:
    def __init__(self,string_tuning_and_number):
        self.generator = InstrumentChordGenerator(tuning=string_tuning_and_number)
        self.flat_to_sharp_mapping = {
            'C': 'C', 'Db': 'C#', 'D': 'D', 'Eb': 'D#', 'E': 'E', 'F': 'F',
            'Gb': 'F#', 'G': 'G', 'Ab': 'G#', 'A': 'A', 'Bb': 'A#', 'B': 'B',
            'Fb': 'E', 'Cb': 'B', 'Bbb': 'A', 'Abb': 'G', 'Gbb': 'F', 'Fbb': 'Eb',
            'Ebb': 'D', 'Dbb': 'C'
        }
        self.sharp_to_flat_mapping = {v: k for k, v in self.flat_to_sharp_mapping.items() if k != v}
        
    def create_scale(self, root: str, scale_type: str) -> List[str]:
        if scale_type == 'major':
            return scales.Major(root).ascending()
        elif scale_type == 'natural_minor':
            return scales.NaturalMinor(root).ascending()
        elif scale_type == 'dorian':
            return scales.Dorian(root).ascending()
        elif scale_type == 'harmonic_minor':
            return scales.HarmonicMinor(root).ascending()
        elif scale_type == 'melodic_minor':
            return scales.MelodicMinor(root).ascending()
        elif scale_type == 'mixolydian':
            return scales.Mixolydian(root).ascending()
        else:
            raise ValueError(f"Unsupported scale type: {scale_type}")

    def create_scale_with_sharps(self, root: str, scale_type: str) -> List[str]:
        scale_notes = self.create_scale(root, scale_type)
        return [self.flat_to_sharp_mapping.get(note, note) for note in scale_notes]

    def get_roman_numeral(self, scale: List[str], chord_root: str, quality: str) -> str:
        roman_numerals = ['I', 'II', 'III', 'IV', 'V', 'VI', 'VII']
        position = scale.index(chord_root)
        roman_numeral = roman_numerals[position]
        return roman_numeral if quality == 'MAJOR' else roman_numeral.lower()

    def generate_chord_data(self, root: str) -> List[Dict]:
        sharp_root = self.flat_to_sharp_mapping.get(root, root)
        qualities = [ChordQuality.MAJOR, ChordQuality.MINOR, ChordQuality.DIMINISHED, ChordQuality.AUGMENTED]
        chord_data = []

        for quality in qualities:
            diagrams = self.generator.generate_chord(sharp_root, quality, full_chord=True, partial_chord=False, inversions=True, open_strings=True, max_fret_span=4)
            for diagram in diagrams:
                chord_info = {
                    "Root": sharp_root,
                    "Quality": quality.name,
                    "ChordInfo": {
                        "Frets": diagram.frets,
                        "Fingers": diagram.fingers,
                        "Mutes": diagram.mutes,
                        "Position": diagram.get_position(),
                        "Inversion": diagram.inversion,
                        "IsPartial": diagram.is_partial,
                        "Notes": [str(note) for note in diagram.notes],
                        "NotesOnStrings": diagram.get_notes_with_strings()
                    }
                }
                chord_data.append(chord_info)

        return chord_data

    def get_scale_data(self, key: str, scale_type: str = 'major', max_variations: int = 2) -> Dict:
        scale = self.create_scale_with_sharps(key, scale_type)

        result = {
            "key": key,
            "scale_type": scale_type,
            "scale": scale,
            "chords": []
        }

        for chord in scale:
            chord_data = self.generate_chord_data(chord)
            updated_variations = []
            
            for variation in chord_data[:max_variations]:
                quality = variation['Quality']
                roman_numeral = self.get_roman_numeral(scale, chord, quality)
                variation['ChordInfo']['Nashville Number'] = roman_numeral
                updated_variations.append(variation)

            result["chords"].append({
                "root": self.sharp_to_flat_mapping.get(chord, chord),
                "variations": updated_variations
            })

        return result

    def process_request(self, event: Dict) -> Dict:
        try:
            params = event.get('queryStringParameters', {})
            key = params.get('key', 'C')
            scale_type = params.get('scale_type', 'major')
            max_variations = int(params.get('max_variations', 2))

            result = self.get_scale_data(key, scale_type, max_variations)

            return {
                'statusCode': 200,
                'body': json.dumps(result),
                'headers': {
                    'Content-Type': 'application/json',
                    'Access-Control-Allow-Origin': '*'
                }
            }
        except Exception as e:
            return {
                'statusCode': 500,
                'body': json.dumps({'error': str(e)}),
                'headers': {
                    'Content-Type': 'application/json',
                    'Access-Control-Allow-Origin': '*'
                }
            }


In [44]:



test_event = {
  "queryStringParameters": {
    "key": "Gb",
    "max_variations": "2",
    "scale_type": "dorian"  }
}


local_generator = LocalChordGenerator(string_tuning_and_number=['A4', 'D4', 'A4', 'E5'])

result = local_generator.process_request(test_event)
print(json.dumps(result, indent=2))


{
  "statusCode": 200,
  "body": "{\"key\": \"Gb\", \"scale_type\": \"dorian\", \"scale\": [\"F#\", \"G#\", \"A\", \"B\", \"C#\", \"D#\", \"E\", \"F#\"], \"chords\": [{\"root\": \"Gb\", \"variations\": [{\"Root\": \"F#\", \"Quality\": \"MAJOR\", \"ChordInfo\": {\"Frets\": [9, 8, -1, 9], \"Fingers\": [1, 1, 0, 1], \"Mutes\": [false, false, true, false], \"Position\": \"Fret 8\", \"Inversion\": 0, \"IsPartial\": false, \"Notes\": [\"F#5\", \"A#4\", \"C#6\"], \"NotesOnStrings\": [\"A4:F#5\", \"D4:A#4\", \"A4:C#6\"], \"Nashville Number\": \"I\"}}, {\"Root\": \"F#\", \"Quality\": \"MAJOR\", \"ChordInfo\": {\"Frets\": [-1, 4, 4, 6], \"Fingers\": [0, 1, 1, 1], \"Mutes\": [true, false, false, false], \"Position\": \"Fret 4\", \"Inversion\": 0, \"IsPartial\": false, \"Notes\": [\"F#4\", \"C#5\", \"A#5\"], \"NotesOnStrings\": [\"A4:F#4\", \"D4:C#5\", \"A4:A#5\"], \"Nashville Number\": \"I\"}}]}, {\"root\": \"Ab\", \"variations\": [{\"Root\": \"G#\", \"Quality\": \"MAJOR\", \"ChordInfo\": {\"Fret